## **Save this file as studentid1_studentid2_lab#.ipynb**
(Your student-id is the number shown on your student card.)

E.g. Hence as an example for 2 students:
12301230_3434343_lab1.ipynb.

**This will be parsed by a regexp, so please double check your filename.**

**Only one member of each group has to submit the file to canvas.**

Before you turn this problem in, please make sure everything runs correctly. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Note, that **you are not allowed to use Google Colab**.

**Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your names and email adresses below.**



In [ ]:
NAME = "Rebecca Davidsson"
NAME2 = "Jurre Wolsink"
EMAIL = "rebeccadavidsson3@gmail.com"
EMAIL2 = "jurre.wolsink@student.uva.nl"

# Lab 1: Linear Regression and Overfitting

### Machine Learning 1, September/October 2020

Notes on implementation:

* You should write your code and answers in this IPython Notebook: http://ipython.org/notebook.html. If you have problems, please contact your teaching assistant.
* Please write your answers right below the questions.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Refer to last week's lab notes, i.e. http://docs.scipy.org/doc/, if you are unsure about what function to use. There are different correct ways to implement each problem!
* For this lab, your regression solutions should be in closed form, i.e., should not perform iterative gradient-based optimization but find the exact optimum directly.
* use the provided test boxes to check if your answers are correct

In [ ]:
%pylab inline
plt.rcParams["figure.figsize"] = [20,10]

In [ ]:
# This cell makes sure that you have all the necessary libraries installed

import sys
import platform
from importlib.util import find_spec, module_from_spec

def check_newer_version(version_inst, version_nec):
    version_inst_split = version_inst.split('.')
    version_nec_split = version_nec.split('.')
    for i in range(min(len(version_inst_split), len(version_nec_split))):
        if int(version_nec_split[i]) > int(version_inst_split[i]):
            return False
        elif int(version_nec_split[i]) < int(version_inst_split[i]):
            return True
    return True


module_list = [('jupyter', '1.0.0'), 
               ('matplotlib', '2.0.2'), 
               ('numpy', '1.13.1'), 
               ('python', '3.6.2'), 
               ('sklearn', '0.19.0'), 
               ('scipy', '0.19.1'), 
               ('nb_conda', '2.2.1')]

packages_correct = True
packages_errors = []

for module_name, version in module_list:
    if module_name == 'scikit-learn':
        module_name = 'sklearn'
    if 'python' in module_name:
        python_version = platform.python_version()
        if not check_newer_version(python_version, version):
            packages_correct = False
            error = f'Update {module_name} to version {version}. Current version is {python_version}.'
            packages_errors.append(error) 
            print(error)
    else:
        spec = find_spec(module_name)
        if spec is None:
            packages_correct = False
            error = f'Install {module_name} with version {version} or newer, it is required for this assignment!'
            packages_errors.append(error) 
            print(error)
        else:
            x = __import__(module_name)
            if hasattr(x, '__version__') and not check_newer_version(x.__version__, version):
                packages_correct = False
                error = f'Update {module_name} to version {version}. Current version is {x.__version__}.'
                packages_errors.append(error) 
                print(error)

try:
    from google.colab import drive
    packages_correct = False
    error = """Please, don't use google colab!
It will make it much more complicated for us to check your homework as it merges all the cells into one."""
    packages_errors.append(error) 
    print(error)
except:
    pass

packages_errors = '\n'.join(packages_errors)

$\newcommand{\bPhi}{\mathbf{\Phi}}$
$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\bt}{\mathbf{t}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bm}{\mathbf{m}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\bI}{\mathbf{I}}$

## Part 1: Polynomial Regression

### 1.1. Generate periodic data (5 points)
Write a method `gen_sine(N)` that generates toy data like in fig 1.2 of Bishop's book. The method should have a parameter $N$, and should return $N$-dimensional vectors $\bx$ and $\bt$, where $\bx$ contains evenly spaced values from 0 to (including) 2$\pi$, and the elements $t_i$ of $\bt$ are distributed according to:

$$t_i \sim \mathcal{N}(\mu_i, \sigma^2)$$

where $x_i$ is the $i$-th elements of $\bf{x}$, the mean $\mu_i = \sin(x_i)$ and the standard deviation $\sigma = 0.25$. You can make use of `np.random.normal()` (Hint: Double check its input parameters).


In [ ]:
import numpy as np


def gen_sine(n):
    
    x = np.linspace(0, 2*np.pi, n)
    t = np.random.normal(np.sin(x), 0.25)
    
    return x, t


In [ ]:
### Test your function
np.random.seed(42)
N = 10
x, t = gen_sine(N)

assert x.shape == (N,), "the shape of x is incorrect"
assert t.shape == (N,), "the shape of t is incorrect"



### 1.2 Polynomial regression (10 points)

Write a method `fit_polynomial(x, t, M)` that finds the maximum-likelihood solution of an _unregularized_ $M$-th order polynomial for some dataset `x`. The error function to minimize w.r.t. $\bw$ is:

$E(\bw) = \frac{1}{2} (\bPhi\bw - \bt)^T(\bPhi\bw - \bt)$

where $\bPhi$ is the _feature matrix_ (or _design matrix_) as explained in Bishop's book at section 3.1.1, $\bt$ is the vector of target values. Your method should return a vector $\bw$ with the maximum-likelihood parameter estimates, as well as the _feature matrix_ $\bPhi$.

In [ ]:
def designmatrix(x, M): 
    """Helper function that computes Phi"""
    return np.vander(x, M+1, increasing=True)

def fit_polynomial(x, t, M):
    """ 
    Solve w_ml = (phi^T * phi)^-1  * phi^T * t
    """
    phi = designmatrix(x, M)
    
    # Calculate the Moore-Penrose pseudo-inverse of the matrix
    # From example 3.17 in Bishop's book
    product = np.matmul(phi.T, phi)
    inverse = np.linalg.inv(product)
    w_ml = np.dot(np.matmul(inverse, phi.T), t)
    
    return w_ml, phi


In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
m = 2
w, Phi = fit_polynomial(x,t,m)

assert w.shape == (m+1,), "The shape of w is incorrect"
assert Phi.shape == (N, m+1), "The shape of Phi is incorrect"



### 1.3 Plot (5 points)
Sample a dataset with $N=10$, and fit four polynomials with $M \in (0, 2, 4, 8)$.
For each value of $M$, plot the prediction function, along with the data and the original sine function. The resulting figure should look similar to fig 1.4 of the Bishop's book. Note that you can use matplotlib's `plt.pyplot(.)` functionality for creating grids of figures.

In [ ]:
N = 10
x, t = gen_sine(N)
n = 2                               # nr of plots
M = [0, 2, 4, 8]                    # degree of polynomial
sin = np.arange(0, 2*np.pi, 0.01)   # original sine function

def f(n, x, c):
    """
    n: degree of polynomial
    x: points to calculate (np.linspace or np.arange)
    c: coefficients calculated by fit_polynomial
    """
    y = 0
    for i in range(n + 1):
        y += c[i] * x **i
    return y


def plot_polynomials(unreguralized=True):
    fig = plt.figure(figsize=(13,10))
    fig.subplots_adjust(hspace=0.2, wspace=0.2)
        
    for degree_index in range(0, 4):
        ax = fig.add_subplot(2, 2, degree_index + 1)
        ax.grid()
        ax.set_xlabel("X")
        ax.set_ylabel("t")
        ax.title.set_text("Dregree = " + str(M[degree_index]))

        # x-axis pi-formatter
        ax.xaxis.set_major_formatter(FuncFormatter(lambda val, pos: '{:.0g}$\pi$'.format(val/np.pi) if val !=0 else '0'))
        ax.xaxis.set_major_locator(MultipleLocator(base=np.pi))

        # data points of gen_sine function
        ax.scatter(x, t, label="Data")
        
        # original sine function
        plt.plot(sin, np.sin(sin), label="Original sine function", color="blue")
    
        # Calculate function for multiple points
        space = np.linspace(0, 2*np.pi, 1000)
        
        # get the coefficients
        if not unreguralized:
            w, Phi = fit_polynomial_reg(x, t, M[degree_index], lamb)
            y = f(M[degree_index], space, w)
            ax.plot(space, y, color="red", label="Polynomial fit reguralized")
    
        w, Phi = fit_polynomial(x, t, M[degree_index])
        y = f(M[degree_index], space, w)
        ax.plot(space, y, color="orange", label="Polynomial fit")
        ax.legend()


    plt.show()

plot_polynomials()


### 1.4 Regularized linear regression (15 points)

a) (10 points) Write a method `fit_polynomial_reg(x, t, M, lamb)` that fits a _regularized_ $M$-th order polynomial to the periodic data, as discussed in the lectures, where `lamb` is the regularization term _lambda_. (Note that 'lambda' cannot be used as a variable name in Python since it has a special meaning). The error function to minimize w.r.t. $\bw$:

$E(\bw) = \frac{1}{2} (\bPhi\bw - \bt)^T(\bPhi\bw - \bt) + \frac{\lambda}{2} \mathbf{w}^T \mathbf{w}$

For background, see section 3.1.4 of Bishop's book.

The function should return $\bw$ and $\bPhi$.

In [ ]:
def fit_polynomial_reg(x, t, m, lamb):
    """ 
    Solve w_ml = (lamda * I + phi^T * phi)^-1  * phi^T * t
    """
    phi = designmatrix(x, m)
    I = np.identity(m + 1)
    product = lamb * I + np.matmul(phi.T, phi)
    inverse = np.linalg.inv(product)
    w_ml = np.dot(np.matmul(inverse, phi.T), t)
    
    return w_ml, phi

N = 10
lamb = 0.1
plot_polynomials(unreguralized=False)



In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
m = 2
lamb = 0.1
w, Phi = fit_polynomial_reg(x,t,m, lamb)

assert w.shape == (m+1,), "The shape of w is incorrect"
assert Phi.shape == (N, m+1), "The shape of w is incorrect" 



b) (5 points) What changes do you notice in $\bw$ and $\bPhi$ after introducing the regularization term? Why is this happening? 

(Write no more than 5 lines. For example, you can consider the simple test case with $t = 0.3*x^2 + 2.5$)

The values for w get smaller. This happens because the error function is now dependent on w and there is a penalty term added to the error function that prevents the values from becoming too large. Phi stays the same.






### 1.5 Model selection by cross-validation (15 points)
Use cross-validation to find a good choice of $M$ and $\lambda$, given a dataset of $N=10$ datapoints generated with `gen_sine(10)`. You should write a function that tries (loops over) a reasonable range of choices of $M$ and $\lambda$, and returns the choice with the best cross-validation error. In this case you use $K=5$ folds.

You can let $M \in (0, 1, ..., 10)$, and let $\lambda \in (e^{-10}, e^{-9}, ..., e^{0})$.

a) (5 points) First of all, write a method `pred_error(x_train, x_valid, t_train, t_valid, M, lamb)` that compares the prediction of your method `fit_polynomial_reg` for a given set of parameters $M$ and $\lambda$ to `t_valid`. It should return the prediction error for a single fold.

In [ ]:
# https://mathworld.wolfram.com/LeastSquaresFittingPolynomial.html

def pred_error(x_train, x_valid, t_train, t_valid, M, reg):
    """Return prediction of one single fold"""
    w, Phi = fit_polynomial_reg(x_train, t_train, M, reg)
    
    # Create new matrix of size M
    A = designmatrix(x_valid, M)
    
    # solve y = wA, where A is the Vandermonde matrix
    y = w.dot(A.T)
    diff = y - t_valid
    pred_err = np.sum(diff**2)

    return pred_err


In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
M = 2
reg = 0.1
pred_err = pred_error(x[:-2], x[-2:], t[:-2], t[-2:], M, reg)

assert pred_err < 0.001, "pred_err is too big"



b) (10 points) Now write a method find_best_m_and_lamb(x, t) that finds the best values for $M$ and $\lambda$. The method should return the best $M$ and $\lambda$. To get you started, here is a method you can use to generate indices of cross-validation folds.

In [ ]:
def kfold_indices(N, k):
    all_indices = np.arange(N,dtype=int)
    np.random.shuffle(all_indices)
    idx = [int(i) for i in np.floor(np.linspace(0,N,k+1))]
    train_folds = []
    valid_folds = []
    for fold in range(k):
        valid_indices = all_indices[idx[fold]:idx[fold+1]]
        valid_folds.append(valid_indices)
        train_folds.append(np.setdiff1d(all_indices, valid_indices))
    return train_folds, valid_folds


In [ ]:
def find_best_m_and_lamb(x, t):
    M_range = np.arange(0, 11, 1)
    reg_range = np.array([np.exp(i-10) for i in range(11)])
    M_best, lamb_best = M_range[0], reg_range[0]
    error = 10e7
    k = 5
    train_folds, valid_folds = kfold_indices(len(x), k)

    for m in M_range:
        for l in reg_range:
            pred_err = 0
            for training, valid in zip(train_folds, valid_folds):
                pred_err += pred_error(x[training], 
                                       x[valid], 
                                       t[training], 
                                       t[valid], m, l)

            # keep storing better values until best is found
            if pred_err < error:
                error = pred_err
                lamb_best = l
                M_best = m

    return M_best, lamb_best

N = 10
x, t = gen_sine(N)
find_best_m_and_lamb(x, t)

In [ ]:
### This is not an empty cell (You don't need to care about it).

### 1.6 Why grid search? (5 points)

Grid search is an commonly-used technique to tune hyper-parameters in a model.
Considering the case described in the previous step of this assignment, running a grid search over the possible parameter values (10 possible values for both $M$ and $\lambda$), results in two nested loops exploring $10 \times 10 = 100$ different configurations for the model. 

a) (3 points) Why do we want to optimize by changing the two hyperparameters at the same time, and not in a sequential way? We could initialise all parameters randomly, fix one parameter at a time and iterate over the other, resulting in only $10 + 10 = 20$ experiments!

If that method was applied, the entire solution space will not be searched. For example, a better solution could have been found at iteration 99, but this iteration would not be performed with this method because not all combinations would have been tested. 

For more complex models, the number of combinations easily explodes with the number of parameters. For example, with 5 parameters we would run $10 \times 10 \times 10 \times 10 \times 10 = 100,000$ experiments.

b) (2 points) Try to think or find in literature one alternative to grid search to tune hyper-parameters more efficiently. Explain very briefly (2-3 lines max) how this method avoids the combinatorial explosion we have see in grid search.

Several algorithms such as Gradient-based optimization, Bayesian optimisation and evolutionary algorithms can be used. For example, Bayesian optimisation in takes into account past evaluations when choosing the hyperparameter set for the next iteration. It focusses on the areas of the parameter space that it believes will bring the best scores.

### 1.7 Plot best cross-validated fit (5 points)

For some dataset with $N = 10$, plot the model with the optimal $M$ and $\lambda$ according to the cross-validation error, using the method you just wrote. In addition, the plot should show the dataset itself and the function that we try to approximate. Let the plot make clear which $M$ and $\lambda$ were found.

In [ ]:
N = 10
x, t = gen_sine(N)

def f(n, x, c):
    """
    n: degree of polynomial
    x: points to calculate (np.linspace or np.arange)
    c: coefficients calculated fit_polynomial
    """
    y = 0
    for i in range(n + 1):
        y += c[i] * x **i
    return y

m, lamb = find_best_m_and_lamb(x, t)

fig = plt.figure(figsize=(13,10))
fig.subplots_adjust(hspace=0.2, wspace=0.2)

ax = fig.add_subplot(2, 2, 1)
ax.grid()
ax.set_xlabel("X")
ax.set_ylabel("t")
ax.title.set_text("Dregree = " + str(m) + ", lambda = " + str(lamb))

# x-axis pi-formatter
ax.xaxis.set_major_formatter(FuncFormatter(lambda val, pos: '{:.0g}$\pi$'.format(val/np.pi) if val !=0 else '0'))
ax.xaxis.set_major_locator(MultipleLocator(base=np.pi))

# data points of gen_sine function
ax.scatter(x, t, label="Data")

# original sine function
plt.plot(sin, np.sin(sin), label="Original sine function", color="blue")

# Calculate function for multiple points
space = np.linspace(0, 2*np.pi, 1000)

w, Phi = fit_polynomial_reg(x, t, m, lamb)
y = f(m, space, w)


ax.plot(space, y, color="red", label="Polynomial fit reguralized")

ax.legend()

plt.show()



## Part 2: Bayesian Linear (Polynomial) Regression

### 2.1 Sine 2 (5 points)

Write a function `gen_sine2(N)` that behaves identically to `gen_sine(N)` except that the generated values $x_i$ are not linearly spaced, but drawn from a uniform distribution between $0$ and $2 \pi$.

In [ ]:
def gen_sine2(n):
    x = np.random.uniform(0, np.pi*2, size=(n,))
    t = np.random.normal(np.sin(x), 0.25)
    
    return x, t

In [ ]:
### Test your function
np.random.seed(42)
N = 10
x, t = gen_sine2(N)

assert x.shape == (N,), "the shape of x is incorrect"
assert t.shape == (N,), "the shape of t is incorrect"



### 2.2 Compute Posterior (15 points)

You're going to implement a Bayesian linear regression model, and fit it to the periodic data. Your regression model has a zero-mean isotropic Gaussian prior over the parameters, governed by a single (scalar) precision parameter $\alpha$, i.e.:

$$p(\bw \;|\; \alpha) = \mathcal{N}(\bw \;|\; 0, \alpha^{-1} \bI)$$

The covariance and mean of the posterior are given by:

$$\bS_N= \left( \alpha \bI + \beta \bPhi^T \bPhi \right)^{-1} $$
$$\bm_N = \beta\; \bS_N \bPhi^T \bt$$

where $\alpha$ is the precision of the predictive distribution, and $\beta$ is the noise precision. 
See MLPR chapter 3.3 for background.

Write a method `fit_polynomial_bayes(x, t, M, alpha, beta)` that returns the mean $\bm_N$ and covariance $\bS_N$ of the posterior for a $M$-th order polynomial. In addition it should return the design matrix $\bPhi$. The arguments `x`, `t` and `M` have the same meaning as in question 1.2.

In [ ]:
def fit_polynomial_bayes(x, t, M, alpha, beta):
    Phi = designmatrix(x, M)
    S = np.linalg.inv(alpha * np.identity(Phi.shape[1]) + beta * np.matmul(Phi.T, Phi))
    m = beta * np.dot(np.matmul(S, Phi.T), t)
    return m, S, Phi


In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
M = 2
alpha = 0.6
beta = 16
m, S, Phi = fit_polynomial_bayes(x, t, M, alpha, beta)

assert m.shape == (M+1,), "the shape of m is incorrect" 
assert S.shape == (M+1, M+1), "the shape of S is incorrect"
assert Phi.shape == (N, M+1), "the shape of Phi is incorrect"



### 2.3 Prediction (10 points)

The predictive distribution of Bayesian linear regression is:

$$ p(t \;|\; \bx, \bt, \alpha, \beta) = \mathcal{N}(t \;|\; \bm_N^T \phi(\bx), \sigma_N^2(\bx))$$

$$ \sigma_N^2 = \frac{1}{\beta} + \phi(\bx)^T \bS_N \phi(\bx) $$

where $\phi(\bx)$ are the computed features for a new datapoint $\bx$, and $t$ is the predicted variable for datapoint $\bx$. 

Write a function that `predict_polynomial_bayes(x, m, S, beta)` that returns the predictive mean, variance and design matrix $\bPhi$ given a new datapoint `x`, posterior mean `m`, posterior variance `S` and a choice of model variance `beta`.

In [ ]:
def predict_polynomial_bayes(x, m, S, beta):
    Phi = designmatrix(x, len(m)-1)
    mean = np.dot(Phi, m)
    sigma = 1 / beta + np.sum(np.dot(Phi, S) * Phi, axis=1)
    return mean, sigma, Phi


In [ ]:
### Test your function
np.random.seed(42)
N = 10
x = np.linspace(-1, 1, N)
m = np.random.rand(3)
S = np.random.rand(3, 3)
beta = 16
mean, sigma, Phi = predict_polynomial_bayes(x, m, S, beta)

assert mean.shape == (N,), "the shape of mean is incorrect"
assert sigma.shape == (N,), "the shape of sigma is incorrect"
assert Phi.shape == (N, m.shape[0]), "the shape of Phi is incorrect"



### 2.4 Plot predictive distribution (10 points)

a) (5 points) Generate 10 datapoints with `gen_sine2(10)`. Compute the posterior mean and covariance for a Bayesian polynomial regression model with $M=4$, $\alpha=\frac{2}{5}$ and $\beta=\frac{1}{0.25^2}$.
Plot the Bayesian predictive distribution, where you plot (for $x$ between 0 and $2 \pi$) $t$'s predictive mean and the predictive standard deviation using `plt.fill_between(..., alpha=0.1)` (the alpha argument induces transparency).

Include the datapoints in your plot.


In [ ]:
M = 4
alpha = 2/5
beta = 1/(0.25)**2
x, t = gen_sine2(10)
m_n, S_n, _ = fit_polynomial_bayes(x, t, M, alpha, beta)

space = np.linspace(0, 2*np.pi, 100)
mean, sigma, _ = predict_polynomial_bayes(space, m_n, S_n, beta)

plt.scatter(x, t, label='Data')
plt.plot(space, np.sin(space), label='Original sine function', color='blue')
plt.plot(space, mean, label='Polynomial fit', color='red')
plt.fill_between(space, mean+np.sqrt(sigma), mean-np.sqrt(sigma), alpha=0.1, color='red')
plt.legend()
plt.grid()
plt.xlabel("X")
plt.ylabel("t")
plt.title("Degree = 4")
plt.show()

b) (5 points) For a second plot, draw 100 samples from the parameters' posterior distribution. Each of these samples is a certain choice of parameters for 4-th order polynomial regression. 
Display each of these 100 polynomials.


In [ ]:
samples = np.random.multivariate_normal(m_n, S_n, (100,))

poly = designmatrix([1], M)[0]

lines = np.matmul(designmatrix(space, M), samples.T)

for line in lines.T:
    plt.plot(space, line)
    
plt.grid()
plt.xlabel("X")
plt.ylabel("t")
plt.title("Degree = 4")
plt.show()

### 2.5 Additional questions (10 points)

a) (5 points) Why is $\beta=16$ the best choice of $\beta$ in section 2.4?



Because of the formula of sigma; this formula is dependent on beta. When N is really large (N=100) it can be found that: 
$$ \sigma^2 = \frac{1}{\beta} $$ 
because beta dominates,
and 
$$ \sigma = \frac{1}{\sqrt{\beta}} = \frac{1}{\sqrt{16}} = 0.25 $$ 

b) (5 points) What problems do we face when it comes to choosing basis functions in linear models?

M basis function along each dimension of a D-dimensional input space requires M^D basis functions: the curse of dimensionality.

Also, the chosen basis function have a big influence on the effectiveness of the model: a more general model will require more data to fit, and different models are more appropriate for different problems. 